Agent-based modeling of microglia

Andrew Gallimore, Enrique Lopez, Sela Raisl, Mack Kona, Anthony Wolfe and Kamila Larripa. 

California Polytechnic University, Humboldt

In [9]:
import mesa
import math
import numpy as np
import pandas as pad
import seaborn as sns

from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import CellAgent, FixedAgent, OrthogonalMooreGrid
from mesa.visualization import CommandConsole, Slider, SolaraViz, SpaceRenderer, make_plot_component
from mesa.visualization.components import AgentPortrayalStyle
from mesa.experimental.devs import ABMSimulator

Agents

In [3]:
class MicrogliaAgent(CellAgent):
    
    def __init__(self, model, phenotype=None, energy=10, stopped=False, immune_tolerance=0, cell=None):
        """
        Initalize a microglia 
        """
        super().__init__(model)
        self.phenotype = phenotype
        self.energy = 10
        self.immune_tolerance = 0
        self.cell = cell
    
    def feed(self):
        """abstract method, TODO"""

    def step(self):
        self.move()
        self.energy -= 1
        self.feed()
        
        if self.energy() < 0:
            self.remove()

class HomeostaticMicroglia(MicrogliaAgent):
    
    def feed(self):
        patch = next(obj for obj in self.cell.agents if isinstance(obj,Nueron))

    def move(self):
        cells_without_microglia = self.cell.neighborhood.select(
            lambda cell: not any(isinstance(obj, HomeostaticMicroglia) for obj in cell.agents)
        )
        cells_with_nuerons = cells_without_microglia.select(
            lambda cell: any(isinstance(obj, Nueron) and obj.alive for obj in cell.agents)
        )
        target_cells = (
            cells_with_nuerons if len(cells_with_nuerons) > 0 else cells_without_microglia
        )
        self.cell = target_cells.select_random_cell()

#lass Proinflammatory(MicrogliaAgent):
 #   x

class Nueron(FixedAgent):

    @property
    def alive(self):
        return self.__alive

    def __init__(self, model, cell):
        super().__init__(model)
        self.cell = cell

In [4]:
class MicrogliaAgent(CellAgent):
    
    def __init__(self, model, phenotype=None, energy=0, stopped=False, immune_tolerance=0, cell=None):
        super().__init__(model)
        self.cell = cell
        self.age = 1
        self.energy = 10




    def move(self):
        self.cell = self.cell.neighborhood.select_random_cell()

    def announceAge(self):
        print(f"Agent {self.unique_id}: {self.age!s}.")

Model Code

In [ ]:
class MicrogliaModel(Model):
    def __init__(
        self,
        width=33,
        height=33,
        initial_microglia=2,
        initial_nuerons=6,
        seed=None,
        simulator:ABMSimulator = None
        ):
        
        super().__init__(seed=seed)
        self.simulator = simulator
        self.simulator.setup(self)

        # init model parameters
        self.height = height
        self.width = width

        # grid
        self.grid = OrthogonalMooreGrid([self.height, self.width], torus=True, capacity=100, random=self.random,)

        # creating reporters
        model_reporters = {
            "Microglia": lambda m: len(m.agents_by_type[HomeostaticMicroglia]),
            "Nueron": lambda m: len(m.agents_by_type[Nueron])
        }

        self.datacollector = DataCollector(model_reporters)

        ## create microglia
        HomeostaticMicroglia.create_agents(self, initial_microglia, phenotype=None, energy=10, stopped=False, immune_tolerance=0, cell=self.random.choices(self.grid.all_cells.cells, k=initial_microglia))
        
        Nueron.create_agents(self, initial_nuerons, phenotype=None, energy=10, stopped=False, immune_tolerance=0, cell=self.random.choices(self.grid.all_cells.cells, k=initial_nuerons))

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.agents_by_type[HomeostaticMicroglia].shuffle_do("step")

        self.datacollector.collect(self)
    

App